In [2]:
import yfinance as yf

# Get stock data
ticker = "AAPL"
data = yf.download(ticker, period="1y")

# Print the data
print(data.info())
print(data)

[*********************100%***********************]  1 of 1 completed

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 250 entries, 2024-06-13 to 2025-06-12
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   (Close, AAPL)   250 non-null    float64
 1   (High, AAPL)    250 non-null    float64
 2   (Low, AAPL)     250 non-null    float64
 3   (Open, AAPL)    250 non-null    float64
 4   (Volume, AAPL)  250 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 11.7 KB
None
Price            Close        High         Low        Open    Volume
Ticker            AAPL        AAPL        AAPL        AAPL      AAPL
Date                                                                
2024-06-13  213.242737  215.741048  210.615026  213.740409  97862700
2024-06-14  211.500870  214.168387  210.316407  212.854540  70122700
2024-06-17  215.661423  217.930808  211.729813  212.376781  93728300
2024-06-18  213.292480  217.612289  212.008492  216.577122  79943300
2024-06-20  208.703964 

In [17]:
stock = yf.Ticker("AAPL")
stock.info

{'address1': 'One Apple Park Way',
 'city': 'Cupertino',
 'state': 'CA',
 'zip': '95014',
 'country': 'United States',
 'phone': '(408) 996-1010',
 'website': 'https://www.apple.com',
 'industry': 'Consumer Electronics',
 'industryKey': 'consumer-electronics',
 'industryDisp': 'Consumer Electronics',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and p

In [12]:
import pandas as pd

df = pd.DataFrame()
df.empty

True

In [ ]:
import os
from typing import Literal

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import StructuredMessage
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from pydantic import BaseModel


# The response format for the agent as a Pydantic base model.
class AgentResponse(BaseModel):
    thoughts: str
    response: Literal["happy", "sad", "neutral"]


# Create an agent that uses the OpenAI GPT-4o model.
model_client = OpenAIChatCompletionClient(
    model="gemini-2.0-flash",
    api_key=os.getenv("GEMINI_API_KEY"),
)
agent = AssistantAgent(
    "assistant",
    model_client=model_client,
    system_message="Categorize the input as happy, sad, or neutral following the JSON format.",
    # Define the output content type of the agent.
    output_content_type=AgentResponse,
)

result = await Console(agent.run_stream(task="I am happy."))

# Check the last message in the result, validate its type, and print the thoughts and response.
assert isinstance(result.messages[-1], StructuredMessage)
assert isinstance(result.messages[-1].content, AgentResponse)
print("res:", result)
print("Thought: ", result.messages[-1].content.thoughts)
print("Response: ", result.messages[-1].content.response)
await model_client.close()

---------- TextMessage (user) ----------
I am happy.
---------- StructuredMessage[AgentResponse] (assistant) ----------
{"thoughts":"The user expressed happiness.","response":"happy"}
res: messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 13, 1, 40, 55, 25109, tzinfo=datetime.timezone.utc), content='I am happy.', type='TextMessage'), StructuredMessage[AgentResponse](source='assistant', models_usage=RequestUsage(prompt_tokens=31, completion_tokens=22), metadata={}, created_at=datetime.datetime(2025, 6, 13, 1, 40, 56, 65210, tzinfo=datetime.timezone.utc), content=AgentResponse(thoughts='The user expressed happiness.', response='happy'), format_string=None, type='StructuredMessage[AgentResponse]')] stop_reason=None
Thought:  The user expressed happiness.
Response:  happy


In [16]:
from autogen_core.models import ModelInfo, UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(
    model="gemini-2.0-flash-lite",
    model_info=ModelInfo(
        vision=True,
        function_calling=True,
        json_output=True,
        family="unknown",
        structured_output=True,
    ),
    api_key=os.getenv("GEMINI_API_KEY"),
)

response = await model_client.create(
    [UserMessage(content="What is the capital of France?", source="user")]
)
print(response)
await model_client.close()


finish_reason='stop' content='The capital of France is Paris.\n' usage=RequestUsage(prompt_tokens=7, completion_tokens=8) cached=False logprobs=None thought=None
